# baichuan2-13b-RE 微调及推理

## 加载模型

In [1]:
import os
os.environ["http_proxy"] = "http://10.6.0.17:8888"
os.environ["https_proxy"] = "http://10.6.0.17:8888"

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## 微调

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python ../src/finetune.py \
    --do_train --do_eval \
    --overwrite_output_dir \
    --model_name_or_path 'baichuan-inc/Baichuan2-13B-Chat' \
    --checkpoint_dir 'zjunlp/baichuan2-13b-iepile-lora' \
    --stage 'sft' \
    --model_name 'baichuan' \
    --template 'baichuan2' \
    --train_file 'iepile-re-train-transformed.json' \
    --valid_file 'iepile-re-val-transformed.json' \
    --output_dir='lora/baichuan13B-iepile-re' \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --gradient_accumulation_steps 4 \
    --preprocessing_num_workers 16 \
    --num_train_epochs 10 \
    --learning_rate 5e-5 \
    --max_grad_norm 0.5 \
    --optim "adamw_torch" \
    --max_source_length 400 \
    --cutoff_len 700 \
    --max_target_length 300 \
    --evaluation_strategy "epoch" \
    --save_strategy "epoch" \
    --save_total_limit 10 \
    --lora_r 64 \
    --lora_alpha 64 \
    --lora_dropout 0.05 \
    --bf16 \
    --bits 4


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following direct

## 推理

In [ ]:
!CUDA_VISIBLE_DEVICES=1 python ../src/inference.py \
    --stage sft \
    --model_name_or_path 'baichuan-inc/Baichuan2-13B-Chat' \
    --checkpoint_dir 'lora/baichuan13B-iepile-re' \
    --model_name 'baichuan' \
    --template 'baichuan2' \
    --do_predict \
    --input_file 'iepile-re-test-transformed.json' \
    --output_file 'results/baichuan2-13b-test-output.json' \
    --finetuning_type lora \
    --output_dir 'lora/oneke-continue-test' \
    --predict_with_generate \
    --cutoff_len 512 \
    --bf16 \
    --max_new_tokens 300 \
    --bits 4



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda117.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following direct

In [7]:
!python ../ie2instruction/eval_func.py \
  --path1 results/baichuan2-13b-test-output.json \
  --task RE

{'total': {'总样本数': 237, '错误数': 0, 'P': 27.45, 'R': 28.64, 'F1': 28.03}}
